In [61]:
import re
import json
from collections import defaultdict
from ordered_set import OrderedSet

from transformers import AutoTokenizer

In [62]:
tokenizer = AutoTokenizer.from_pretrained("kanishka/smolm-autoreg-bpe-seed_111")

In [77]:
# read lexicon/adaptation.json and generalization.json
with open("../data/lexicon/adaptation.json") as f:
    adaptation = json.load(f)
with open("../data/lexicon/generalization.json") as f:
    generalization = json.load(f)

In [78]:
adaptation_vocab = OrderedSet()
for k, v in adaptation.items():
    # adaptation_vocab.add(k)
    # adaptation_vocab.update(v)
    for vv in v:
        vv = re.sub(r'(the\s|little|some|of|red|beautiful|big)+', '', vv).strip()
        adaptation_vocab.add(vv)


generalization_vocab = OrderedSet()
for k, v in generalization.items():
    # generalization_vocab.add(k)
    # generalization_vocab.update(v)
    for vv in v:
        vv = re.sub(r'(the\s|little|some|of|red|beautiful|big|small|bit|a\s|few|cute|piece)+', '', vv).strip()
        generalization_vocab.add(vv)

In [79]:
generalization_vocab

OrderedSet(['him', 'them', 'we', 'us', 'he', 'it', 'they', 'cookie monster', 'daddy', 'lucy', 'grandma', 'david', 'barney', 'nemo', 'pooh', 'puppy', 'baby', 'cat', 'book', 'milk', 'soup', 'cake', 'cheerios', 'toys', 'car'])

In [80]:
# adaptation_vocab
generalization_vocab

OrderedSet(['him', 'them', 'we', 'us', 'he', 'it', 'they', 'cookie monster', 'daddy', 'lucy', 'grandma', 'david', 'barney', 'nemo', 'pooh', 'puppy', 'baby', 'cat', 'book', 'milk', 'soup', 'cake', 'cheerios', 'toys', 'car'])

In [81]:
# corpus
corpus = []
with open("../../smolm/data/corpora/babylm_data/babylm_100M/aochildes.train", "r") as f:
    for line in f:
        corpus.append(line.strip())

In [82]:
# calculate all the single word frequencies in the corpus
word_freq = defaultdict(lambda: 0)
for line in corpus:
    # words = line.split()
    words = tokenizer.tokenize(line)
    for word in words:
        word = word.replace("Ġ", "")
        word_freq[word] += 1

In [86]:
adaptation_freq = defaultdict(int)

for word in adaptation_vocab:
    if " " not in word:
        # print(word, word_freq[word.lower()])
        adaptation_freq[word] = word_freq[word.lower()]
    else:
        count = 0
        for line in corpus:
            if word in line:
                count += 1
        # print(word, count)
        adaptation_freq[word] = count

adaptation_freq = dict(sorted(adaptation_freq.items(), key=lambda item: item[1], reverse=True))

In [87]:
adaptation_freq

{'it': 91720,
 'I': 72646,
 'me': 17834,
 'she': 14997,
 'mommy': 8675,
 'her': 7592,
 'ball': 3777,
 'bear': 2665,
 'bird': 2099,
 'dog': 1898,
 'juice': 1782,
 'doll': 677,
 'chocolate': 629,
 'teddy': 401,
 'grandpa': 396,
 'candy': 357,
 'elmo': 328,
 'kitty': 319,
 'bert': 229}

In [88]:
generalization_freq = defaultdict(int)

for word in generalization_vocab:
    if " " not in word:
        # print(word, word_freq[word.lower()])
        generalization_freq[word] = word_freq[word.lower()]
    else:
        count = 0
        for line in corpus:
            if word in line:
                count += 1
        # print(word, count)
        generalization_freq[word] = count

generalization_freq = dict(sorted(generalization_freq.items(), key=lambda item: item[1], reverse=True))

generalization_freq

{'it': 91720,
 'we': 33200,
 'he': 26888,
 'they': 16206,
 'them': 9131,
 'baby': 6830,
 'him': 6742,
 'daddy': 6644,
 'book': 5305,
 'car': 4135,
 'cat': 2435,
 'us': 2404,
 'milk': 1911,
 'toys': 1470,
 'grandma': 717,
 'cake': 717,
 'puppy': 708,
 'soup': 418,
 'pooh': 357,
 'david': 279,
 'cookie monster': 227,
 'cheerios': 106,
 'lucy': 103,
 'nemo': 69,
 'barney': 66}

In [75]:
word_freq['monster']

465

In [76]:
count = 0
for line in corpus:
    if "cookie monster" in line:
        count += 1
count

227